## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [1]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd

In [2]:
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [3]:
def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

In [ ]:
## test for individual run

In [4]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct.json') as inf:
    query1 = json.load(inf)

In [5]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:entity_negatively_regulates_entity"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:SmallMolecule"
                    ]
                }
            }
        }
    }
}


In [7]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=f6e7a7c8-20ee-4ef5-a5f1-e5204e087744


In [ ]:
result_status = retrieve_ars_results(kcresult)

In [ ]:
result_status 

In [ ]:
pwd

In [ ]:
df

In [ ]:
def make_hyperlink(value):
    #url = "https://custom.url/{}"
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [ ]:
make_hyperlink('https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482')

In [ ]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [ ]:
df

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
df